### Занятие №11: Немного чистой архитектуры. Тестирование.

##### <span style="color:#0ab49a">Задача:</span> смоделировать функции, вычисляющие RSSI с одной полуволновой антенны на другую 
- Формула передачи Фрииса: $$\frac{P_r}{P_t}=G_t G_r \left( \frac{\lambda}{4\pi R} \right)^2$$
- Коэффициент усиления полуволнового диполя: *(Silver S. Microwave Antenna Theory and Design // McGraw Hill Book Company. 1984. pp. 98–99.)* $$G=\frac{\cos\left(\frac{\pi}{2}\cos\theta\right)}{\sin\theta}$$

##### SOLID
1. <u>Принцип единственной ответственности</u><br>
        У каждого класса только 1 ответственность; у него нет других обязанностей
2. <u>Принцип открытости/закрытости</u><br>
        Сущности (классы, модули, функции) октрыты для расширения, но закрыты для изменений
3. <u>Принцип подстановки Лисков</u><br>
        Функции, которые используют базовый тип, должны иметь возможность использовать подтипы базового типа, не зная об этом.
4. <u>Принцип разделения интерфейса</u><br>
        Ни один клиент не должен зависеть от методов, которые он не использует
5. <u>Принцип инверсии зависимостей</u><br>
        Абстракции не должны зависеть от деталей. Детали должны зависеть от абстракций.

##### Структура кода

1. 

##### Отображение

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def plot_model_gain(get_RSSI, d: Device, n: int = 20):
    fig = plt.figure(figsize=(15, 10))
    ax = fig.add_subplot(1, 1, 1, projection='3d')

    u = np.linspace(0, 2 * np.pi, n)
    v = np.linspace(-np.pi / 2, np.pi / 2, n)
    U, V = np.meshgrid(u, v)

    g = np.array([[get_gain(...) for ii in range(n)] for jj in range(n)])
    
    X, Y, Z = pol2dec(g, U, V)
    ax.plot_surface(X, Y, Z)
    ax.set_box_aspect([1, 1, 1])
    ax.set_title("Принятый сигнал")
    plt.show()
    
plot_model_gain(get_RSSI, d)

##### Тестирование (unittest / pytest)

**Пример:**

```Python
import pytest

def setup_module(module):
    #init_something()
    pass

def teardown_module(module):
    # teardown_something()
    pass

def test_upper():
    assert 'foo'.upper() == 'FOO'
    
def test_isupper():
    assert 'FOO'.isupper()
    
def test_failed_upper():
    assert 'foo'.upper() == 'FOo'
    
if __name__ == '__main__':
    test_upper()
```

##### Вопрос

Что надо изменить, чтобы помимо мощности принимаемого сигнала измерять ещё и фазу?

##### <span style="color:#0ab49a">Примечание:</span> Кватернионы в ПО

**Инициализация**

In [ ]:
import numpy as np
import quaternion

for q in [np.quaternion(1, 0, 0, 0),
          np.quaternion(1, 2, 3, 4),
          np.quaternion(5, 0, 0),
          np.quaternion(5),
          np.quaternion()]:
    print(f"q = {q}")

q = np.quaternion(1, 2, 3, 4)

In [ ]:
a = np.array([0, 0, 1])

q1 = quaternion.from_vector_part(a)
q2 = quaternion.from_euler_angles(a)

print(f"Кватернион из вектора: {q1}")
print(f"Кватернион из углов Эйлера: {q2}")

**Преобразования типов**

In [ ]:
a = quaternion.as_float_array(q)
b = quaternion.np.array(q)
c = quaternion.np.asarray(q)
d = q.components

print(f"Как numpy-массив: {a}")
print(type(a))
print(f"Как numpy-массив: {b}")
print(type(b))
print(f"Как numpy-массив: {c}")
print(type(c))
print(f"Как numpy-массив: {d}")
print(type(d))

In [ ]:
a = np.array([0, 0, 0.5])

q1 = quaternion.from_vector_part(a)
q2 = quaternion.from_euler_angles(a)

q3 = np.quaternion(np.sqrt(1 - np.linalg.norm(a)**2), *a)

print(f"Кватернион из вектора: {q1}")
print(f"Кватернион единичный из вектора: {q3}")
print(f"Кватернион из углов Эйлера: {q2}")

**Преобразовани математические**

In [ ]:
a = q.real
b = q.vec
w = q.w
x = q.x
y = q.y
z = q.z

print(f"Скалярная часть: {a}")
print(type(a))
print(f"Векторная часть: {b}")
print(type(b))
print(f"Составлющие: {w} {x} {y} {z}")
print(type(w), type(x), type(y), type(z))

In [ ]:
a = quaternion.np.normalized(q)

print(f"Нормированный кватернион: {a}")
print(type(a))

In [ ]:
a = quaternion.np.abs(q)
b = quaternion.np.absolute(q)
d = q.abs()

c = quaternion.np.norm(q)

print(f"Модуль: {a} | {b} | {d}")
print(type(a), type(b), type(d), '\n')

print(f"Норма: {c} (√c = {np.sqrt(c)})")
print(type(c))

In [ ]:
a = quaternion.np.angle_of_rotor(q)
b = q.angle()

print(f"Угол поворота: {a} | {b}")
print(type(a), type(b))

In [ ]:
a = q.conj()
b = q.conjugate()

с = q.inverse()
d = q.reciprocal()

print(f"Сопряженный: {a} | {b}")
print(type(a), type(b), '\n')

print(f"Обратный: {с} | {d}")
print(type(с), type(d))

In [ ]:
A = quaternion.as_rotation_matrix(q)
B = quaternion.as_rotation_matrix(q.normalized())

print(f"Матрица поворота из кватерниона A: \n{A}")
print(f"Матрица поворота из кватерниона B: \n{B}")